In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
with open('/Users/alex/Desktop/files.txt', 'r') as f: 
    files = f.readlines()

In [3]:
files = [file.strip('\n').replace('/pool/data/globus/', '') for file in files]

In [4]:
for file in files: 
    
    fileparts = file.split('/')
    if fileparts[0].split('_')[1] != 'genomics': print(file)
    if fileparts[1].split('_')[0] != 'CGND': print(file)
        
    if fileparts[2][:7] == 'Sample_':
        
        if not ('.fastq.gz' in fileparts[3] or 
                 '.bam' in fileparts[3] or 
                 '.bai' in fileparts[3] or 
                 '.cram' in fileparts[3] or 
                 '.crai' in fileparts[3] or 
                 '.haplotypeCalls.er.raw.vcf.gz' in fileparts[3] or 
                 '.haplotypeCalls.er.raw.vcf.gz.tbi' in fileparts[3] or 
                 '.recalibrated.haplotypeCalls.vcf.gz' in fileparts[3] or 
                 '.recalibrated.haplotypeCalls.vcf.gz.tbi' in fileparts[3] or 
                 '.recalibrated.haplotypeCalls.annotated.vcf.gz' in fileparts[3] or 
                 '.recalibrated.haplotypeCalls.annotated.vcf.gz.tbi' in fileparts[3] or 
                 '.recalibrated.haplotypeCalls.annotated.clinical.txt' in fileparts[3] or 
                 '.recalibrated.haplotypeCalls.annotated.coding.txt' in fileparts[3] or 
                 '.recalibrated.haplotypeCalls.annotated.coding_rare.txt' in fileparts[3] or 
                 '.recalibrated.haplotypeCalls.annotated.txt' in fileparts[3] or                 
                 '.genomestrip.del.bed' in fileparts[3] or
                 '.eh.vcf' in fileparts[3]): 
            print(file)
    
    elif fileparts[2] == 'StructuralVariants': continue

    elif fileparts[2] == 'ExpansionHunter': continue
        
    else: print(file)

In [5]:
dd = defaultdict(set)

for file in files: 
    fileparts = file.split('/')
    project = fileparts[0]
    quote = fileparts[1]
    sample = fileparts[2]

    dd[project+'/'+quote+'/'+sample].add(fileparts[3])

In [6]:
d = dict()
for sample, files in dict(dd).items():
    
    d[sample] = {
        'fastq': len([file for file in files if '.fastq.gz' in file]),
        'bam': len([file for file in files if '.final.bam' in file or '.final.bai' in file]),
        'cram': len([file for file in files if '.final.cram' in file or '.final.crai' in file]),
        'gVCF': len([file for file in files if '.raw.vcf' in file]),
        'VCF': len([file for file in files if 'haplotypeCalls.vcf.gz' in file]),
        'annotation': len([file for file in files if '.annotated.' in file]),
        'SVariants': len([file for file in files if '.genomestrip.del.bed' in file]),
        'ExpansionHunter VCF': len([file for file in files if '.eh.vcf' in file])
    }

In [7]:
df = pd.DataFrame(d).reindex(['fastq','bam','cram','gVCF','VCF','annotation','SVariants', 'ExpansionHunter VCF']).T
df.head()

,fastq,bam,cram,gVCF,VCF,annotation,SVariants,ExpansionHunter VCF
aals_genomics/CGND_12001/Sample_CGND-HDA-00187,16,0,2,2,2,6,1,0
aals_genomics/CGND_12001/Sample_CGND-HDA-00202,16,0,2,2,2,6,1,0
aals_genomics/CGND_12001/Sample_CGND-HDA-00203,16,0,2,2,2,6,1,0
aals_genomics/CGND_12001/Sample_CGND-HDA-00204,16,0,2,2,2,6,1,0
aals_genomics/CGND_12001/Sample_CGND-HDA-00205,16,0,2,2,2,6,1,0


In [8]:
df['project'], df['quote'], df['sample'] = df.index.str.split('/').str
df = df.set_index(['project','quote','sample'])
df.head()

fastq  bam  cram  gVCF  VCF  \
project       quote      sample                                               
aals_genomics CGND_12001 Sample_CGND-HDA-00187     16    0     2     2    2   
                         Sample_CGND-HDA-00202     16    0     2     2    2   
                         Sample_CGND-HDA-00203     16    0     2     2    2   
                         Sample_CGND-HDA-00204     16    0     2     2    2   
                         Sample_CGND-HDA-00205     16    0     2     2    2   

                                                annotation  SVariants  \
project       quote      sample                                         
aals_genomics CGND_12001 Sample_CGND-HDA-00187           6          1   
                         Sample_CGND-HDA-00202           6          1   
                         Sample_CGND-HDA-00203           6          1   
                         Sample_CGND-HDA-00204           6          1   
                         Sample_CGND-HDA-00205           6          1   

                                                ExpansionHunter VCF  
project       quote      sample                                      
aals_genomics CGND_12001 Sample_CGND-HDA-00187                    0  
                         Sample_CGND-HDA-00202                    0  
                         Sample_CGND-HDA-00203                    0  
                         Sample_CGND-HDA-00204                    0  
                         Sample_CGND-HDA-00205                    0

In [9]:
df.to_csv('/Users/alex/Desktop/df.csv')